# <center> Procesamiento de datos a refined </center>

## Librerías

In [1]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F, Window
from pyspark.sql.types import IntegerType, DoubleType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1761950954788_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Funciones

In [2]:
def limpiar_nombre(nombre):
    if nombre is None:
        return "Desconocido"
    # Reemplazar cualquier carácter que no sea letra o número por "_"
    return re.sub(r'[^A-Za-z0-9_]', '_', nombre.strip())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Lectura de datos

In [3]:
spk = SparkSession.builder \
    .appName("trusted-to-refined-spark") \
    .getOrCreate()

print("Spark iniciado:", spk)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark iniciado: <pyspark.sql.session.SparkSession object at 0x7fa214172e50>

In [4]:
path_trusted = "s3://s3-pi-perfilamiento-20252/trusted/"
df = spark.read.option("header", "true").csv(path_trusted)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------+-------------------+-----+--------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal|Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia| estado|OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|  ciudad|    tipoVinculacion|Edad|Tipo_Empresa|     tomador|         nomProducto|nomGrupoEmpresarial| Ramo|orden_vigencia|
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------+-------------------+-----+--------------+
|        37|       69|            Anual|  642179.0|             642179.0|         2023-03-01|      2026-03-01|Vigente|     Reno

## Procesamiento de datos para el modelo

In [6]:
# Crear ventana por cliente, ordenando por la fecha de inicio
window_spec = Window.partitionBy("codCliente").orderBy(F.asc("fechaInicioVigencia"))

# Crear columna orden_poliza: asigna el mismo número a pólizas con igual fecha
df_final = df.withColumn("orden_poliza", F.dense_rank().over(window_spec))

df_final.orderBy("codCliente", "orden_poliza").show(100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-----------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+------------------------------------------------------+-------------------+-------------+--------------+------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal |Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado     |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad              |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto                                           |nomGrupoEmpresarial|Ramo         |orden_vigencia|orden_poliza|
+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-----------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+-----------------------------

In [7]:
# Ordenar y numerar las pólizas por cliente y ramo según la fecha más antigua
w_ramo = Window.partitionBy("codCliente", "Ramo").orderBy(F.col("fechaInicioVigencia").asc())

df_final = df_final.withColumn(
    "rn_ramo",
    F.row_number().over(w_ramo)
)

# Filtrar para quedarnos solo con la primera póliza (más antigua) por ramo y cliente
df_final = df_final.filter(F.col("rn_ramo") == 1).drop("rn_ramo")

# Volver a recalcular orden_poliza ahora con las pólizas restantes
w_poliza = Window.partitionBy("codCliente").orderBy(F.col("fechaInicioVigencia").asc())

df_final = df_final.withColumn("orden_poliza", F.dense_rank().over(w_poliza))

# Resultado ordenado para revisar visualmente
df_final.orderBy("codCliente", "orden_poliza").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+----------------------------------+-------------------+-----+--------------+------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal |Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad              |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto                       |nomGrupoEmpresarial|Ramo |orden_vigencia|orden_poliza|
+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+----------------------------------+-------------------+-----+--------------+------------+
|10000 

In [8]:
# Ventana por cliente, ordenada por la secuencia de las pólizas
w = Window.partitionBy("codCliente").orderBy("orden_poliza")

# Crear lista de ramos anteriores a cada fila
df_final = df_final.withColumn(
    "ramos_previos",
    F.collect_list("Ramo").over(w.rowsBetween(Window.unboundedPreceding, -1))
)

# Reemplazar null por lista vacía
df_final = df_final.withColumn(
    "ramos_previos",
    F.when(F.col("ramos_previos").isNull(), F.array()).otherwise(F.col("ramos_previos"))
)

# Crear columna objetivo Y (siguiente ramo que compró el cliente)
df_final = df_final.withColumn(
    "Y",
    F.lead("Ramo").over(w)
)

# Obtener lista de ramos únicos y sus nombres limpios
ramos_unicos = [r["Ramo"] for r in df_final.select("Ramo").distinct().collect() if r["Ramo"] is not None]
ramos_unicos_limpios = [limpiar_nombre(r) for r in ramos_unicos]

# Crear columnas binarias (1 si ya lo compró antes O si es el actual)
for ramo_original, ramo_col in zip(ramos_unicos, ramos_unicos_limpios):
    df_final = df_final.withColumn(
        ramo_col,
        F.when(
            (F.array_contains(F.col("ramos_previos"), F.lit(ramo_original))) |
            (F.col("Ramo") == ramo_original),
            1
        ).otherwise(0)
    )

# Eliminar columna auxiliar
df_final = df_final.drop("ramos_previos")

df_final.orderBy("codCliente", "orden_poliza").select(
    "codCliente", "orden_poliza", *ramos_unicos_limpios, "Y"
).show(50, truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+-------------+-----+----+------------+-----+-----+-----+
|codCliente|orden_poliza|Patrimoniales|Otros|Vida|Cumplimiento|Autos|Salud|Y    |
+----------+------------+-------------+-----+----+------------+-----+-----+-----+
|10000     |1           |0            |0    |1   |0           |0    |0    |Autos|
|10000     |2           |0            |0    |1   |0           |1    |0    |NULL |
|100001    |1           |0            |0    |0   |0           |0    |1    |NULL |
|100003    |1           |0            |0    |1   |0           |0    |0    |Salud|
|100003    |2           |0            |0    |1   |0           |0    |1    |NULL |
|100004    |1           |0            |0    |0   |0           |0    |1    |NULL |
|100005    |1           |0            |0    |0   |0           |0    |1    |NULL |
|100007    |1           |0            |0    |1   |0           |0    |0    |NULL |
|100010    |1           |0            |0    |1   |0           |0    |0    |NULL |
|100012    |1   

In [9]:
df_final = df_final.filter(F.col("Y").isNotNull())
print("Número de registros:", df_final.count())

df_cliente = df_final.filter(F.col("codCliente") == "1500")
df_cliente.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N?mero de registros: 9353
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+-----------+-------------------+----+--------------+------------+-----+-------------+-----+----+------------+-----+-----+
|codCliente|codPoliza|formaPagoVigencia|valorTotal|Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad  |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto|nomGrupoEmpresarial|Ramo|orden_vigencia|orden_poliza|Y    |Patrimoniales|Otros|Vida|Cumplimiento|Autos|Salud|
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+-----------+-------------------+----+---------

In [10]:
#Conservar solo las columnas necesarias
df_final = df_final.select(
    "codCliente",
    "codPoliza",
    "Ramo",
    "Y",
    "Salud",
    "Vida",
    "Autos",
    "Cumplimiento",
    "Patrimoniales",
    "Otros"
)
df_final.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-------------+-------------+-----+----+-----+------------+-------------+-----+
|codCliente|codPoliza|         Ramo|            Y|Salud|Vida|Autos|Cumplimiento|Patrimoniales|Otros|
+----------+---------+-------------+-------------+-----+----+-----+------------+-------------+-----+
|     10004|    14919|        Autos|        Salud|    0|   0|    1|           0|            0|    0|
|     10031|   211649|        Autos|Patrimoniales|    0|   0|    1|           0|            0|    0|
|     10038|    15246|        Salud|        Autos|    1|   0|    0|           0|            0|    0|
|    100391|    63849|Patrimoniales|        Autos|    0|   0|    0|           0|            1|    0|
|    100442|    64753| Cumplimiento|        Autos|    0|   0|    0|           1|            0|    0|
|    100447|   127299|Patrimoniales|         Vida|    0|   0|    0|           0|            1|    0|
|    100504|    96897|         Vida|Patrimoniales|    0|   1|    0|           0|           

## Guardar datos en refined

In [11]:
path_refined = "s3://s3-pi-perfilamiento-20252/refined/"

df_final.coalesce(1) \
        .write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv(path_refined)

print(f"Datos de la zona refined guardados exitosamente en: {path_refined}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Datos de la zona refined guardados exitosamente en: s3://s3-pi-perfilamiento-20252/refined/